## Creating a cached database

This notebook will show you how to create a cached database from the full LSST CatSim database at UW.

To do this you will have to have the LSST sims packages already setup and a connection to UW available. To set up the connection see [here](https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database).

In [ ]:
import numpy as np
import os
from lsst.utils import getPackageDir
from lsst.sims.utils import ObservationMetaData
from lsst.sims.catalogs.db import fileDBObject
from lsst.sims.catUtils.baseCatalogModels import StarObj, SNDBObj
from desc.twinkles import create_galaxy_cache

### Set up the center of the field you want to cache and the radius of the field.

In [ ]:
_obs = ObservationMetaData(pointingRA=53.0091385,
                           pointingDec=-27.4389488,
                           boundType='circle',
                           boundLength=3.)

### Specify the columns you want to grab from the database.

The CatSim schema is found [here](https://confluence.lsstcorp.org/display/SIM/Database+Schema).

In [ ]:
star_dtype = np.dtype([('simobjid', int),
                        ('ra', float), ('decl', float),
                        ('magNorm', float),
                        ('mura', float), ('mudecl', float),
                        ('parallax', float), ('ebv', float),
                        ('vrad', float), ('varParamStr', '|S256'),
                        ('sedfilename', '|S40'),
                        ('umag', float), ('gmag', float), ('rmag', float), ('imag', float),
                        ('zmag', float), ('ymag', float)])

### Specify the cache database name and the location of a temporary text file that we will use to write out and read in to sqlite.

In [ ]:
col_names = list(star_dtype.names)
star_cache_name = os.path.join(getPackageDir('twinkles'), 'data',
                                   'twinkles_star_cache.txt')
star_db_name = os.path.join(getPackageDir('twinkles'), 'data',
                                'star_cache.db')

### Open the database connection

In [ ]:
db = StarObj(database='LSSTCATSIM',
             host='fatboy.phys.washington.edu',
             port=1433, driver='mssql+pymssql')

### Run the database query and save temporarily to a text file

In [ ]:
result_iterator = db.query_columns(colnames=col_names, chunk_size=100000,
                                    obs_metadata=_obs)
with open(star_cache_name, 'w') as output_file:
    output_file.write('# ')
    for name in col_names:
        output_file.write('%s ' % name)
    output_file.write('\n')
    for chunk in result_iterator:
        for line in chunk:
            output_file.write(('%d;%.17g;%.17g;%.17g;%.17g;%.17g;%.17g;%.17g;%.17g;%s;%s;%.17g;%.17g;%.17g;%.17g;%.17g;%.17g\n' %
                                  (line[1], line[2], line[3], line[4], line[5], line[6], line[7],
                                   line[8], line[9], str(line[10]), str(line[11]),line[12],line[12],line[13],
                                   line[14],line[15],line[16])).replace('nan','NULL'))

### Load the objects into the database

In [ ]:
if os.path.exists(star_db_name):
    os.unlink(star_db_name)

In [ ]:
dbo = fileDBObject(star_cache_name, driver='sqlite', runtable='star_cache_table',
                    database=star_db_name, dtype=star_dtype, delimiter=';',
                    idColKey='simobjid')